In [2]:
import os
import datasets
import re
from datasets import load_dataset

/home/exx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
names_dataset = [('Idavidrein/gpqa', 'gpqa_main'), ('Salesforce/wikitext', 'wikitext-2-raw-v1'), ('openai/gsm8k', 'main')]
ds = load_dataset(*names_dataset[2])['test']
print(ds)

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})


In [6]:
def parse_lines_with_index(pat, lines, index=0, target_indent=0):
    mydoc = {'texts': [], 'subdocs': []}

    while index < len(lines):
        line = lines[index]
        m = pat.match(line)
        if m:
            left_indent = m.group("left").count("=")
            if left_indent < target_indent:
                break   # same return value in exit condition
            elif left_indent == target_indent:
                if len(mydoc['texts']) == 0:
                    mydoc['texts'].append(line.lstrip().rstrip())
                    index += 1
                    continue
                else:   # hit a new-same indent
                    break
                # end
            else: # left_indent > target_indent(cannot be the same)
                subdoc, index = parse_lines_with_index(pat, lines, index, left_indent)
                mydoc['subdocs'].append(subdoc)
            # end
        else:
            if len(line) != 0:
                mydoc['texts'].append(line.lstrip().rstrip())
            # end
            index += 1
            continue
        # end
    # end

    return mydoc, index
# end


In [7]:
import json
pat = re.compile(r'^\s*(?P<left>(?:=\s*)+)\s*(?P<text>[^=\n]*?)\s*(?P<right>(?:=\s*)+)\s*$')

mydoc, _ = parse_lines_with_index(pat, ds)

In [14]:
def merge_subdocs(subdocs) -> tuple[list[str], list[str]]:
    lines = []
    titles = []

    for subdoc in subdocs:
        lines += subdoc['texts']
        titles.append(subdoc['texts'][0])

        sublines, subtitles = merge_subdocs(subdoc['subdocs'])
        lines += sublines
        titles += subtitles
    # end for
    
    return lines, titles
# end


In [18]:
lines, titles = merge_subdocs(mydoc['subdocs'][0]['subdocs'])

['= = Career = =',
 '= = = 2000 – 2005 = = =',
 'In 2000 Boulter had a guest @-@ starring role on the television series The Bill ; he portrayed " Scott Parry " in the episode , " In Safe Hands " . Boulter starred as " Scott " in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . A review of Boulter \'s performance in The Independent on Sunday described him as " horribly menacing " in the role , and he received critical reviews in The Herald , and Evening Standard . He appeared in the television series Judge John Deed in 2002 as " Addem Armitage " in the episode " Political Expediency " , and had a role as a different character " Toby Steele " on The Bill .',
 'He had a recurring role in 2003 on two episodes of The Bill , as character " Connor Price " . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . Boulter s